Check simple clustering and also the constrained kmeans.

To run the script, move it to the root dir.

In [7]:
import pandas as pd 
import numpy as np 
import new_utils
from typing import Dict
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from scipy.stats import mode
from copkmeans.cop_kmeans import cop_kmeans

In [8]:
def compute_theta_phi_for_biomarker(data, biomarker, data_source, max_attempt = 50, seed = None):
    """get theta and phi parameters for this biomarker using constrained k-means
    input: 
        - biomarker_df: a pd.dataframe of a specific biomarker
    output: 
        - a tuple: theta_mean, theta_std, phi_mean, phi_std
    """
    if seed is not None:
        # Set the seed for numpy's random number generator
        rng = np.random.default_rng(seed)
    else:
        rng = np.random 

    # Filter data for the current biomarker
    # reset_index is necessary here because we will use healthy_df.index later
    biomarker_df = data[data['biomarker']== biomarker].reset_index(drop=True)
    n_clusters = 2
    measurements = np.array(biomarker_df['measurement']).reshape(-1, 1)
    healthy_df = biomarker_df[biomarker_df['diseased'] == False]

    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    predictions = clustering.fit_predict(measurements)
    
    # Verify that AgglomerativeClustering produced exactly 2 clusters with more than 1 member each
    cluster_counts = np.bincount(predictions) # array([3, 2])
    if len(cluster_counts) != n_clusters or any(c <= 1 for c in cluster_counts):
        print("AgglomerativeClustering did not yield the required clusters, switching to KMeans.")
        
        # If AgglomerativeClustering fails, attempt KMeans with a max_attempt limit
        curr_attempt = 0
        n_init_value = 10
        clustering_setup = KMeans(n_clusters=n_clusters, n_init=n_init_value)
        
        while curr_attempt < max_attempt:
            clustering_result = clustering_setup.fit(measurements)
            predictions = clustering_result.labels_
            cluster_counts = np.bincount(predictions) # array([3, 2])
            
            if len(cluster_counts) == n_clusters and all(c > 1 for c in cluster_counts):
                break 
            curr_attempt += 1
        else:
            print(f"KMeans failed. Try randomizing the predictions")
            predictions = rng.choice([0, 1], size=len(measurements))
            cluster_counts = np.bincount(predictions)
            if len(cluster_counts) != n_clusters or not all(c > 1 for c in cluster_counts):
                # Raise an error if a valid clustering isn't found within max_attempt
                raise ValueError(f"KMeans clustering failed to find valid clusters within max_attempt.")
    
    healthy_predictions = predictions[healthy_df.index]
    mode_result = mode(healthy_predictions, keepdims=False).mode
    phi_cluster_idx = mode_result[0] if isinstance(mode_result, np.ndarray) else mode_result
    theta_cluster_idx = 1 - phi_cluster_idx

    # two empty clusters to strore measurements
    clustered_measurements = [[] for _ in range(2)]
    # Store measurements into their cluster
    for i, prediction in enumerate(predictions):
        clustered_measurements[prediction].append(measurements[i][0])
    
     # Calculate means and standard deviations
    theta_mean, theta_std = np.mean(
        clustered_measurements[theta_cluster_idx]), np.std(
            clustered_measurements[theta_cluster_idx])
    phi_mean, phi_std = np.mean(
        clustered_measurements[phi_cluster_idx]), np.std(
            clustered_measurements[phi_cluster_idx])
    
    # Check for invalid values
    if any(np.isnan(v) or v == 0 for v in [theta_std, phi_std, theta_mean, phi_mean]):
        # print(f"One of the calculated values is invalid (0 or NaN) in {biomarker} of {data_source}")
        raise ValueError("One of the calculated values is invalid (0 or NaN).")

    return theta_mean, theta_std, phi_mean, phi_std

In [9]:
# def compute_theta_phi_for_biomarker_cop(data, biomarker, data_source, max_attempt = 100):
#     """get theta and phi parameters for this biomarker using constrained k-means
    
#     https://github.com/Behrouz-Babaki/COP-Kmeans

#     input: 
#         - biomarker_df: a pd.dataframe of a specific biomarker
#     output: 
#         - a tuple: theta_mean, theta_std, phi_mean, phi_std
#     """
#     biomarker_df = data[data['biomarker']== biomarker].reset_index(drop=True)
#     n_clusters = 2
#     measurements = np.array(biomarker_df['measurement']).reshape(-1, 1)
#     healthy_df = biomarker_df[biomarker_df['diseased'] == False]
#     must_link = [(x, x + 1) for x in healthy_df.index[:-1]]

#     curr_attempt = 0
#     while curr_attempt < max_attempt:
#         clusters, centers = cop_kmeans(
#             dataset=measurements, k=n_clusters, ml=must_link)
#         predictions = np.array(clusters)
#         healthy_predictions = predictions[healthy_df.index]
#         cluster_counts = np.bincount(predictions)
#         if all(
#             c > 1 for c in cluster_counts) and len(
#                 cluster_counts) == n_clusters and len(
#                     set(healthy_predictions)) == 1:
#             break 
#         curr_attempt += 1

#     if len(cluster_counts) != n_clusters or not all(c > 1 for c in cluster_counts):
#         print(f"KMeans clustering failed to find valid clusters within max_attempt in {biomarker} of {data_source}")
    
#     phi_cluster_idx = healthy_predictions[0]
#     theta_cluster_idx = 1 - phi_cluster_idx

#     # two empty clusters to strore measurements
#     clustered_measurements = [[] for _ in range(2)]
#     # Store measurements into their cluster
#     for i, prediction in enumerate(predictions):
#         clustered_measurements[prediction].append(measurements[i][0])
    
#      # Calculate means and standard deviations
#     theta_mean, theta_std = np.mean(
#         clustered_measurements[theta_cluster_idx]), np.std(
#             clustered_measurements[theta_cluster_idx])
#     phi_mean, phi_std = np.mean(
#         clustered_measurements[phi_cluster_idx]), np.std(
#             clustered_measurements[phi_cluster_idx])
    
#     # Check for invalid values
#     if any(np.isnan(v) or v == 0 for v in [theta_std, phi_std, theta_mean, phi_mean]):
#         print(f"One of the calculated values is invalid (0 or NaN) in {biomarker} of {data_source}")

#     return theta_mean, theta_std, phi_mean, phi_std

In [10]:
def get_theta_phi_estimates(
    data_source: str,
) -> Dict[str, Dict[str, float]]:
    """
    Obtain theta and phi estimates (mean and standard deviation) for each biomarker.

    Args:
    data (pd.DataFrame): DataFrame containing participant data with columns 'participant', 
        'biomarker', 'measurement', and 'diseased'.
    # biomarkers (List[str]): A list of biomarker names.

    Returns:
    Dict[str, Dict[str, float]]: A dictionary where each key is a biomarker name,
        and each value is another dictionary containing the means and standard deviations 
        for theta and phi of that biomarker, with keys 'theta_mean', 'theta_std', 'phi_mean', 
        and 'phi_std'.
    """
    # empty hashmap of dictionaries to store the estimates
    data = pd.read_csv(f'data/{data_source}.csv')
    estimates = {}
    biomarkers = data.biomarker.unique()
    for biomarker in biomarkers:
        theta_mean, theta_std, phi_mean, phi_std = compute_theta_phi_for_biomarker(
            data, biomarker, data_source
        )
        estimates[biomarker] = {
            'theta_mean': theta_mean,
            'theta_std': theta_std,
            'phi_mean': phi_mean,
            'phi_std': phi_std
        }
    return estimates

In [11]:
j_values = [50, 200, 500]
r_values = [0.1, 0.25, 0.5, 0.75, 0.9]
m_values = range(50)  # From 0 to 49 (inclusive)
# Generate all combinations
data_sources = [f"{int(j*r)}|{j}_{m}" for j in j_values for r in r_values for m in m_values]

In [12]:
for data_source in data_sources:
    get_theta_phi_estimates(data_source)
    print(f"{data_source} is done")

5|50_0 is done
5|50_1 is done
5|50_2 is done
5|50_3 is done
5|50_4 is done
5|50_5 is done
5|50_6 is done
5|50_7 is done
5|50_8 is done
5|50_9 is done
5|50_10 is done
5|50_11 is done
5|50_12 is done
5|50_13 is done
5|50_14 is done
5|50_15 is done
5|50_16 is done
5|50_17 is done
5|50_18 is done
5|50_19 is done
5|50_20 is done
5|50_21 is done
5|50_22 is done
5|50_23 is done
AgglomerativeClustering did not yield the required clusters, switching to KMeans.
5|50_24 is done
5|50_25 is done
5|50_26 is done
5|50_27 is done
5|50_28 is done
5|50_29 is done
5|50_30 is done
5|50_31 is done
5|50_32 is done
5|50_33 is done
5|50_34 is done
5|50_35 is done
5|50_36 is done
5|50_37 is done
5|50_38 is done
5|50_39 is done
5|50_40 is done
5|50_41 is done
5|50_42 is done
5|50_43 is done
5|50_44 is done
5|50_45 is done
5|50_46 is done
5|50_47 is done
5|50_48 is done
5|50_49 is done
12|50_0 is done
12|50_1 is done
12|50_2 is done
12|50_3 is done
12|50_4 is done
12|50_5 is done
12|50_6 is done
12|50_7 is done
